## Machine Hack Buyer's Time Prediction - Feature Engineering

Buyers spend a significant amount of time surfing an e-commerce store, since the pandemic the e-commerce has seen a boom in the number of users across the domains. In the meantime, the store owners are also planning to attract customers using various algorithms to leverage customer behavior patterns.

Tracking customer activity is also a great way of understanding customer behavior and figuring out what can actually be done to serve them better. Machine learning and AI has already played a significant role in designing various recommendation engines to lure customers by predicting their buying patterns.

We are provided with the visitor's session data and are required to create a regression algorithm to predict the time a buyer will spend on the platform.

### Attribute Description
- `session_id`: Unique identifier for every row
- `session_number`: Session type identifier
- `client_agent`: Client-side software details
- `device_details`: Client-side device details
- `date`: Datestamp of the session
- `purchased`: Binary value for any purchase done
- `added_in_cart`: Binary value for cart activity
- `checked_out`:  Binary value for checking out successfully
- `time_spent`: Total time spent in seconds (Target Column)

Here the analysis will be focused on creating features, particularly (and most importantly) based on `client_agent` feature.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Import Packages

In [ ]:
import warnings; warnings.filterwarnings('ignore')
import numpy as np, pandas as pd, re, requests
from sklearn import preprocessing

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

### Read data

In [ ]:
## Read train and test data

train =  pd.read_csv('/kaggle/input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Train.csv', low_memory = False ,parse_dates = ['date'])
test = pd.read_csv('/kaggle/input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Test.csv', low_memory = False ,parse_dates = ['date'])

print(f'Shape of train data: {train.shape}')
print(f'Shape of test data: {test.shape}')

In [ ]:
## Create a column with split identifier

train['split_ID'] = 'Train'; test['split_ID'] = 'Test'
all_df = pd.concat([train, test])

In [ ]:
print(f'Shape of combined dataframe: {all_df.shape}')
display(all_df.head())

### Adding datetime features
Add datetime features such as `year`, `quarter`, `weekofyear`, `month`, `day`, `hour`, `minute`, `dayofyear`, `dayofweek` and `is_season`. Values in `season` are like 1, 2, 3, 4. For e.g. 1 represents Dec, Jan and Feb and 2 is March, April and May. 

Quick reference: `{1:'DJF', 2: 'MAM', 3:'JJA', 4:'SON'}`

In [ ]:
all_df['year'] = pd.to_datetime(all_df['date']).dt.year
all_df['quarter'] = pd.to_datetime(all_df['date']).dt.quarter
all_df['weekofyear'] = pd.to_datetime(all_df['date']).dt.weekofyear
all_df['month'] = pd.to_datetime(all_df['date']).dt.month
all_df['day'] = pd.to_datetime(all_df['date']).dt.day
all_df['hour'] = pd.to_datetime(all_df['date']).dt.hour
all_df['minute'] = pd.to_datetime(all_df['date']).dt.minute
all_df['dayofyear'] = pd.to_datetime(all_df['date']).dt.dayofyear
all_df['dayofweek'] = pd.to_datetime(all_df['date']).dt.dayofweek
all_df['is_weekend'] = np.where(all_df['dayofweek'].isin([6, 0]), 1, 0)
all_df['season'] = ((all_df['month'] % 12 + 3) // 3)

### Adding a column to store rows where `client_agent` is null

In [ ]:
## Create a new column for cases where `client_agent` is null
all_df['ca_isna'] = np.where(all_df['client_agent'].isnull(), 1, 0)

### Column with count of `client_agent` and `device_details` by date

In [ ]:
all_df = all_df.fillna('')
ca_bydate_count = (all_df
                   .fillna('')
                   .groupby(['client_agent', 
                             'device_details',
                             'split_ID', 
                             'date'])
                   .size()
                   .to_frame('ca_bydate_count')
                   .reset_index())

all_df = all_df.merge(ca_bydate_count, 
                      on = ['client_agent',
                            'device_details',
                            'split_ID', 
                            'date'],
                      how = 'left')

display(all_df[['client_agent', 'device_details', 'ca_bydate_count']].drop_duplicates())

### Add features based on `client_agent`
1. Clean `client_agent` with `to_repl` dictionary values
2. To achieve the above we will make use of a helper function
3. Create new column for each category of replace we did

In [ ]:
## Helper function for replacing values
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

## Helper function to create columns after replacement is done
def get_col(col_type):
    all_df['ca_{}'.format(col_type)] = all_df['ca_updated'].apply(
    lambda x: re.findall('{}//.*?\\\\'.format(col_type), x, flags = re.DOTALL))
    all_df['ca_{}'.format(col_type)] = [' '.join(x) for x in list(all_df['ca_{}'.format(col_type)])]
    all_df['ca_{}'.format(col_type)] = (all_df['ca_{}'.format(col_type)]
                        .str.replace('\\', '')
                        .str.replace('{}//'.format(col_type), '')
                        .str.strip())
    return all_df['ca_{}'.format(col_type)]

In [ ]:
to_repl = {'Product/': '\\\Product//',
           'Mozilla/': '\\\Mozilla//',
           'AppleWebKit/': '\\\AppleWebKit//',
           'Chrome/': '\\\Chrome//',
           'Safari/': '\\\Safari//',
           'Mobile/': '\\\Mobile//',
           'iPhone/': '\\\iPhone//',
           'Android/': '\\\Android//',
           'Carrier/': '\\\Carrier//',
           'Type/': '\\\Type//',
           'Device/': '\\\Device//',
           'Gecko/': '\\\Gecko//',
           'iPad/': '\\\iPad//',
           'https://': 'https:/',
           'Typhoeus -': '\\\Typhoeus//',
           'Firefox/': '\\\Firefox//',
           'Opera/': '\\\Opera//',
           'Presto/': '\\\Presto//',
           'CFNetwork/': '\\\CFNetwork//',
           'Darwin/': '\\\Darwin//',
           'CriOS/': '\\\CriOS//',
           'Dalvik/': '\\\Dalvik//',
           'GSA/': '\\\GSA//'}

all_df['ca_updated'] = all_df['client_agent'].apply(lambda x: replace_all(x, to_repl))
all_df['ca_updated'] = all_df['ca_updated'] + '\\\\'
all_df['ca_Product'] = get_col('Product')
all_df['ca_Mozilla'] = get_col('Mozilla')
all_df['ca_AppleWebKit'] = get_col('AppleWebKit')
all_df['ca_Chrome'] = get_col('Chrome')
all_df['ca_Safari'] = get_col('Safari')
all_df['ca_Mobile'] = get_col('Mobile')
all_df['ca_iPhone'] = get_col('iPhone')
all_df['ca_Android'] = get_col('Android')
all_df['ca_Carrier'] = get_col('Carrier')
all_df['ca_Type'] = get_col('Type')
all_df['ca_Device'] = get_col('Device')
all_df['ca_Gecko'] = get_col('Gecko')
all_df['ca_iPad'] = get_col('iPad')
all_df['ca_Typhoeus'] = get_col('Typhoeus')
all_df['ca_Firefox'] = get_col('Firefox')
all_df['ca_Opera'] = get_col('Opera')
all_df['ca_Presto'] = get_col('Presto')
all_df['ca_CFNetwork'] = get_col('CFNetwork')
all_df['ca_Darwin'] = get_col('Darwin')
all_df['ca_CriOS'] = get_col('CriOS')
all_df['ca_Dalvik'] = get_col('Dalvik')
all_df['ca_GSA'] = get_col('GSA')
all_df['ca_Chrome'] = all_df['ca_Chrome'].str.replace('Mobile', '')

In [ ]:
## Dropping `ca_updated` column
all_df.drop(['ca_updated'], axis = 1, inplace = True)

## Value counts of parsed columns
new_cols = list(all_df.select_dtypes(include = 'O').filter(like = 'ca_', axis = 1).columns)
# new_cols = new_cols[3:]
for col in new_cols:
    print('Value counts in {}:'.format(col)) 
    display(pd.DataFrame([all_df[col].value_counts(ascending = False).head(20), all_df[col].value_counts(normalize = True, ascending = False).head(20)], index = ['VC', 'Norm_VC']).T)

### Further cleaning those `ca_*` columns added
1. `ca_Product`: Replacing *'.'* and get first element, replace NAs with UNK
2. `ca_AppleWebKit`: Split by *'/'* and get first element and then replace *' (KHTML, like Gecko)'*, *'(KHTML, like Gecko)'* and *'version'* with blanks. Split by *'.'* and get first element, replace NAs with UNK. Here NAs were converted to *'nan'* when we used `.astype(str)` method.
3. `ca_Mozilla`: Most of the rows have a pattern and use those to clean this column further.
4. `ca_Chrome` and `ca_Safari`: Take only first two part of the versions captured in these columns.
5. `ca_Mobile` and `ca_iPhone`: These again have a pattern i.e. one is available when other is not and yes there are rows where both of these are nulls as well. Added a column to capture that.




In [ ]:
## ca_Product
all_df['ca_Product'] = all_df['ca_Product'].str.split('.').str[0].fillna('UNK')
print('Value counts in ca_Product')
display(all_df['ca_Product'].value_counts(dropna = False))

In [ ]:
## ca_AppleWebKit
all_df['ca_AppleWebKit'] = (all_df['ca_AppleWebKit'].astype(str)
                            .str.split('/')
                            .str[0]
                            .str.replace(' (KHTML, like Gecko)','', regex = False)
                            .str.replace('(KHTML, like Gecko)', '', regex = False)
                            .str.lower().str.replace('version', '')
                            .str.strip()
                            .str.split('.')
                            .str[0]
                            .str.replace('nan', 'UNK'))
print('Value counts in ca_AppleWebKit')
display(all_df['ca_AppleWebKit'].value_counts(dropna = False))

In [ ]:
# ca_Mozilla
suff = []
for row in list(all_df['ca_Mozilla'].fillna('')):
    if (('5.0' in row.lower()) and ('windows nt' in row.lower())):
        suff.append('5.0 Windows NT')
    elif (('5.0' in row.lower()) and ('iphone' in row.lower())):
        suff.append('5.0 iPhone')
    elif (('5.0' in row.lower()) and ('mac os' in row.lower())):
        suff.append('5.0 Mac OS')
    else:
        suff.append('UNK')
all_df['ca_Mozilla'] = pd.Series(suff)

print('Value counts in ca_Mozilla')
display(all_df['ca_Mozilla'].value_counts(dropna = False))

In [ ]:
# ca_Chrome and ca_Safari
all_df['ca_Chrome'] = all_df['ca_Chrome'].str[:2].str.strip().fillna('UNK')
all_df['ca_Safari'] = all_df['ca_Safari'].str.split('.').str[0].fillna('UNK')

print('Value counts in ca_Chrome and ca_Safari')
display(all_df['ca_Chrome'].value_counts(dropna = False), all_df['ca_Safari'].value_counts(dropna = False))

In [ ]:
## ca_Mobile and ca_iPhone
all_df.loc[(all_df['ca_Mobile'] != '') & (all_df['ca_Mobile'].str.len() < 6), 'ca_Mobile'] = all_df['ca_Mobile'].str.zfill(6)
all_df['ca_Mobile'] = all_df['ca_Mobile'].str[:6].str.strip()
all_df.loc[all_df['ca_iPhone'] != '', 'ca_iPhone'] = all_df['ca_iPhone'].str[:1].str.strip()

all_df['ca_Mobile_iPhone'] = np.NaN
all_df.loc[(all_df['ca_iPhone'] != '') & (all_df['ca_Mobile'] == ''), 'ca_Mobile_iPhone'] = all_df['ca_iPhone']
all_df.loc[(all_df['ca_iPhone'] == '') & (all_df['ca_Mobile'] != ''), 'ca_Mobile_iPhone'] = all_df['ca_Mobile']
all_df['ca_Mobile_iPhone'] = all_df['ca_Mobile_iPhone'].fillna('UNK')
del all_df['ca_Mobile'], all_df['ca_iPhone']

print('Value counts in ca_Mobile_iPhone column created')
display(all_df['ca_Mobile_iPhone'].value_counts(dropna = False))

In [ ]:
display(all_df.head())

### Label Encoding
Columns label encoded
- `ca_Product`, 
- `ca_Mozilla` 
- `ca_AppleWebKit` 
- `ca_Chrome` 
- `ca_Safari` 
- `ca_Android` 
- `ca_Carrier` 
- `ca_Type` 
- `ca_Device` 
- `ca_Gecko` 
- `ca_iPad` 
- `ca_Typhoeus` 
- `ca_Firefox` 
- `ca_Opera` 
- `ca_Presto` 
- `ca_CFNetwork` 
- `ca_Darwin` 
- `ca_CriOS` 
- `ca_Dalvik` 
- `ca_GSA` 
- `ca_Mobile_iPhone` 
- `device_details`

We would be also dropping these columns from all_df dataframe

In [ ]:
le_cols = list(all_df.select_dtypes(include = 'O').filter(regex = 'ca_', axis = 1).columns)
le_cols.extend(['device_details'])

for col in le_cols:
    print(f'Label Encoded Col: {col}')
    LE = preprocessing.LabelEncoder()
    all_df[col + '_enc_le'] = LE.fit_transform(all_df[col].astype(str))
    display(pd.DataFrame([all_df[col], all_df[col + '_enc_le']], index = ['Values', 'LE Values']).T.drop_duplicates())
    all_df.drop(col, axis = 1, inplace = True)

 ### Get a view of dataframe after feature engineering

In [ ]:
print(f'Shape of combined dataframe after feature engineering: {all_df.shape}')
display(all_df.head())

### Here's a quick recap
We added:
- Datetime features
- Column to store rows where client_agent is null
- Column with count of client_agent and device_details by date
- Features based on client_agent
And done:
- Label encode string columns

LB Position with these features: Private: 39, Public: 37 (in top 10%)

Please UPVOTE 👍